In [1]:
import numpy as np
from scipy.spatial import Delaunay
from scipy.interpolate import LinearNDInterpolator
from scipy.constants import mu_0
from scipy.constants import elementary_charge as q_e
from scipy.constants import proton_mass as m_i
from astropy.convolution import convolve, convolve_fft
from scipy.signal import fftconvolve
from scipy.interpolate import SmoothBivariateSpline
import write_canonical_flux_tube_quantities as wcf
reload(wcf)

from datetime import date
from datetime import datetime
import visit_writer

import structured_3d_vtk as struc_3d
reload(struc_3d)

import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style('white')

import os

import ion_current_to_mach_number as ic_to_mach
reload(ic_to_mach)

import read_from_sql

from mpl_toolkits.mplot3d import Axes3D

In [2]:
now = datetime.now().strftime("%Y-%m-%d-%H-%M")
out_dir = '../output/' + now
try:
    os.makedirs(out_dir)
except:
    pass

# Write all idl quantities to files

In [3]:
bx_all_planes = wcf.save_idl_quantity_to_unstructured_grids('bx', 'B_x', now,
                                                            x_min=-0.032, x_max=0.028,
                                                            y_min=-0.022, y_max=0.032, 
                                                            z_min=0.249, z_max=0.416)

In [4]:
by_all_planes = wcf.save_idl_quantity_to_unstructured_grids('by', 'B_y', now,
                                                            x_min=-0.032, x_max=0.028,
                                                            y_min=-0.022, y_max=0.032, 
                                                            z_min=0.249, z_max=0.416)

In [5]:
bz_all_planes = wcf.save_idl_quantity_to_unstructured_grids('bz', 'B_z', now,
                                                            x_min=-0.032, x_max=0.028,
                                                            y_min=-0.022, y_max=0.032, 
                                                            z_min=0.249, z_max=0.416)

In [6]:
te_all_planes = wcf.save_idl_quantity_to_unstructured_grids('te', 'T_e', now, 
                                                            x_min=-0.026, x_max=0.028,
                                                            y_min=-0.03, y_max=0.028, 
                                                            z_min=0.249, z_max=0.416,
                                                            bounds=(1e-3, 1e3))

In [7]:
n_all_planes = wcf.save_idl_quantity_to_unstructured_grids('n', 'n', now,
                                                           x_min=-0.026, x_max=0.028,
                                                           y_min=-0.03, y_max=0.028, 
                                                           z_min=0.249, z_max=0.416,
                                                           bounds=(1e3, 1e22))

In [8]:
n_three_planes = wcf.remove_plane(0.302, n_all_planes)

# Normalize Temperature by plane

In [14]:
(x_min, x_max, 
 y_min, y_max,
 z_min, z_max) = wcf.joint_mach_bdot_tp_extent()

spatial_increment = 0.001
mesh = np.meshgrid(np.linspace(x_min, x_max, np.ceil((x_max-x_min)/spatial_increment)),
                   np.linspace(y_min, y_max, np.ceil((y_max-y_min)/spatial_increment)),
                   np.linspace(z_min, z_max, np.ceil((z_max-z_min)/spatial_increment)))

mesh_wo_edges = wcf.remove_edges_mesh([np.array(mesh[0]), 
                                       np.array(mesh[1]), 
                                       np.array(mesh[2])])
ones = np.ones(mesh_wo_edges[0].shape)

time_point = 0
te_interpolator = te_interpolators[time_point]
n_interpolator = n_interpolators[time_point]
    
temperature = wcf.scalar_on_mesh(te_interpolator, mesh_wo_edges)
density = wcf.scalar_on_mesh(n_interpolator, mesh_wo_edges)

In [21]:
maxes = np.nanmax(np.nanmax(temperature, axis=0), axis=0)

In [23]:
temperature.shape

(36, 44, 166)

In [22]:
maxes.shape

(166,)

In [27]:
(temperature / maxes[None, None, :]).shape

(36, 44, 166)

# Prepare Mach probe data 

In [11]:
timesteps = 250

database = '/home/jensv/rsx/jens_analysis/shots_database/source/shots.db'
table = 'Shots'
z_direction_1, z_direction_2 = 0, 180
y_direction_1, y_direction_2 = 90, 270
angle_signs = {0: 1,
               180: -1,
               90: -1,
               0: 1}
min_spectral_density = 1.6e-8

condition_z_0416 = ("campaigns = 'mach_probe_plane_campaign_1'"
                    " AND fiducial_pre_crowbar_gyration_spectral_density > "
                    + str(min_spectral_density) +
                    " AND mach_signals_exist = 1"
                    " AND (mach_orientation = " + str(z_direction_1) +
                    " OR mach_orientation = " + str(z_direction_2) + ")")

condition_y_0416 = ("campaigns = 'mach_probe_plane_campaign_1'"
                    " AND fiducial_pre_crowbar_gyration_spectral_density > "
                    + str(min_spectral_density) +
                    " AND mach_signals_exist = 1"
                    " AND (mach_orientation = " + str(y_direction_1) +
                    " OR mach_orientation = " + str(y_direction_2) + ")")

cursor, connection = read_from_sql.cursor_with_rows(condition_z_0416,
                                                    database,
                                                    table)
z_0416_shots = cursor.fetchall()
cursor.close()
connection.close()

cursor, connection = read_from_sql.cursor_with_rows(condition_y_0416,
                                                    database,
                                                    table)
y_0416_shots = cursor.fetchall()
cursor.close()
connection.close()

condition_z_302 = ("campaigns = 'mach_probe_plane_campaign_2'"
                   " AND fiducial_pre_crowbar_gyration_spectral_density > "
                   + str(min_spectral_density) +
                   " AND mach_signals_exist = 1"
                   " AND (mach_orientation = " + str(z_direction_1) +
                   " OR mach_orientation = " + str(z_direction_2) + ")")

cursor, connection = read_from_sql.cursor_with_rows(condition_z_302,
                                                    database,
                                                    table)
z_0302_shots = cursor.fetchall()
cursor.close()
connection.close()

mach_z_0416_measurements = ic_to_mach.run_mach_analysis(z_0416_shots,
                                                        timesteps, 
                                                        angle_signs) 
mach_y_0416_measurements = ic_to_mach.run_mach_analysis(y_0416_shots,
                                                        timesteps, 
                                                        angle_signs)
mach_z_0302_measurements = ic_to_mach.run_mach_analysis(z_0302_shots,
                                                        timesteps, 
                                                        angle_signs)

mach_z_0416_measurements['delays'] = np.arange(timesteps)
mach_y_0416_measurements['delays'] = np.arange(timesteps)
mach_z_0302_measurements['delays'] = np.arange(timesteps)

mach_z_0416_measurements = struc_3d.average_duplicate_points(mach_z_0416_measurements)
mach_y_0416_measurements = struc_3d.average_duplicate_points(mach_y_0416_measurements)
mach_z_0302_measurements = struc_3d.average_duplicate_points(mach_z_0302_measurements)

mach_y_measurements = {0.416: mach_y_0416_measurements}
mach_z_measurements = {0.302: mach_z_0302_measurements,
                       0.416: mach_z_0416_measurements}

mach_y_all_planes = wcf.save_quantity_to_unstructured_grids(mach_y_measurements, 
                                                            'Mach_y', 'Mach_y', '2016-07-26',
                                                            planes=[0.416],
                                                            x_min=-0.052, x_max=0.052,
                                                            y_min=-0.022, y_max=0.032, 
                                                            z_min=0.249, z_max=0.416,
                                                            bounds=(-10, 10))

mach_z_all_planes = wcf.save_quantity_to_unstructured_grids(mach_z_measurements, 
                                                            'Mach_z', 'Mach_z', '2016-07-26',
                                                            planes=[0.302, 0.416],
                                                            x_min=-0.032, x_max=0.032,
                                                            y_min=-0.022, y_max=0.032, 
                                                            z_min=0.249, z_max=0.416,
                                                            bounds=(-10, 10))
mach_y_all_planes = wcf.remove_nan_points(mach_y_all_planes)
mach_z_all_planes = wcf.remove_nan_points(mach_z_all_planes)

TreeException: %TREE-E-FOPENR, Error opening file read-only.

# Determine derivatives and write to files

In [12]:
bx_triangulation, bx_interpolators = wcf.give_delaunay_and_interpolator(bx_all_planes)
by_triangulation, by_interpolators = wcf.give_delaunay_and_interpolator(by_all_planes)
bz_triangulation, bz_interpolators = wcf.give_delaunay_and_interpolator(bz_all_planes)
te_triangulation, te_interpolators = wcf.give_delaunay_and_interpolator(te_all_planes)
n_triangulation, n_interpolators = wcf.give_delaunay_and_interpolator(n_all_planes)
#mach_y_triangulation, mach_y_interpolators = wcf.give_delaunay_and_interpolator(mach_y_all_planes)
#mach_z_triangulation, mach_z_interpolators = wcf.give_delaunay_and_interpolator(mach_z_all_planes)

n_three_triangulation, n_three_interpolators = wcf.give_delaunay_and_interpolator(n_three_planes)

## Examine planes

In [ ]:
(x_min, x_max, 
 y_min, y_max,
 z_min, z_max) = wcf.joint_mach_bdot_tp_extent()
spatial_increment = 0.001
mesh = np.meshgrid(np.linspace(x_min, x_max, np.ceil((x_max-x_min)/spatial_increment)),
                   np.linspace(y_min, y_max, np.ceil((y_max-y_min)/spatial_increment)))
wcf.plot_planes([0.249, 0.302, 0.357, 0.416], mesh, n_three_interpolators[9])

In [ ]:
mesh[1].shape

# Interpolate mach and temperature in plane calculate ion velocity

In [ ]:
(x_min, x_max, 
 y_min, y_max,
 z_min, z_max) = wcf.joint_mach_bdot_tp_extent()

spatial_increment = 0.001
mesh = np.meshgrid(np.linspace(x_min, x_max, np.ceil((x_max-x_min)/spatial_increment)),
                   np.linspace(y_min, y_max, np.ceil((y_max-y_min)/spatial_increment)),
                   np.linspace(z_min, z_max, np.ceil((z_max-z_min)/spatial_increment)))

mach_y_interpolator = mach_y_interpolators[0]
mach_z_interpolator = mach_z_interpolators[0]
te_interpolator = te_interpolators[0]
 
mach_y = wcf.scalar_on_mesh(mach_y_interpolator, mesh[:2])
mach_z = wcf.scalar_on_mesh(mach_z_interpolator, mesh)
te = wcf.scalar_on_mesh(te_interpolator, mesh)

u_i_y = np.sqrt(te*q_e/m_i)*mach_y
u_i_z = np.sqrt(te*q_e/m_i)*mach_z

u_i_y = np.reshape(u_i_y, mesh[0].shape)
u_i_z = np.reshape(u_i_z, mesh[0].shape)

u_i_y = wcf.remove_edges_scalar_quantity_meshes(u_i_y)
u_i_z = wcf.remove_edges_scalar_quantity_meshes(u_i_z)

# Fit $\alpha$

In [ ]:
alpha = 1
filter_width = 15

(x_min, x_max, 
 y_min, y_max,
 z_min, z_max) = wcf.joint_mach_bdot_tp_extent()

spatial_increment = 0.001
mesh = np.meshgrid(np.linspace(x_min, x_max, np.ceil((x_max-x_min)/spatial_increment)),
                   np.linspace(y_min, y_max, np.ceil((y_max-y_min)/spatial_increment)),
                   np.linspace(z_min, z_max, np.ceil((z_max-z_min)/spatial_increment)))

mesh_wo_edges = wcf.remove_edges_mesh([np.array(mesh[0]), 
                                       np.array(mesh[1]), 
                                       np.array(mesh[2])])
ones = np.ones(mesh_wo_edges[0].shape)

time_point = 200

bx_interpolator = bx_interpolators[time_point]
by_interpolator = by_interpolators[time_point]
bz_interpolator = bz_interpolators[time_point]
te_interpolator = te_interpolators[time_point]
n_interpolator = n_interpolators[time_point]

bx_derivative = wcf.triangulate_derivatives(mesh, bx_triangulation, bx_interpolator, 
                                            increment=0.0000001)
bx_derivative = wcf.remove_edges_derivative_meshes(bx_derivative)
by_derivative = wcf.triangulate_derivatives(mesh, by_triangulation, by_interpolator, 
                                            increment=0.0000001)
by_derivative = wcf.remove_edges_derivative_meshes(by_derivative)
bz_derivative = wcf.triangulate_derivatives(mesh, bz_triangulation, bz_interpolator, 
                                            increment=0.0000001)
bz_derivative = wcf.remove_edges_derivative_meshes(bz_derivative)


current = wcf.current_on_mesh([bx_derivative, 
                               by_derivative, 
                               bz_derivative])
b_field, b_field_norm = wcf.b_field_on_mesh([bx_interpolator, 
                                             by_interpolator,
                                             bz_interpolator], 
                                            mesh_wo_edges, 
                                            bias=2e-2)

temperature = wcf.scalar_on_mesh(te_interpolator, mesh_wo_edges)
density = wcf.scalar_on_mesh(n_interpolator, mesh_wo_edges)

current = np.asarray(current)
density = np.asarray(density)
b_field_norm = np.asarray(b_field_norm)

density = wcf.boxcar_filter_quantity_mesh(density, filter_width)

for direction in xrange(len(current)):
    current[direction] = wcf.boxcar_filter_quantity_mesh(current[direction], filter_width)

density_constant = 1e18*np.ones(density.shape)

ion_velocity_term_1 = wcf.calc_ion_velocity_term_1(current, density, q_e)
ion_velocity_term_1_constant_density = wcf.calc_ion_velocity_term_1(current, density_constant, q_e)
ion_velocity_term_2 = wcf.calc_ion_velocity_term_2(b_field_norm, alpha)

ion_vorticity_term_1 = wcf.calc_ion_vorticity_term_1(current, density, q_e, mesh_wo_edges)
ion_vorticity_term_1_constant_density = wcf.calc_ion_vorticity_term_1(current, density_constant, q_e, mesh_wo_edges)
ion_vorticity_term_2 = wcf.calc_ion_vorticity_term_2(b_field_norm, alpha, mesh_wo_edges)

mesh_wo_edges = wcf.remove_edges_mesh([np.array(mesh[0]), 
                                       np.array(mesh[1]), 
                                       np.array(mesh[2])])

In [ ]:
np.allclose(np.reshape(mesh_wo_edges[0][:,:,-1].ravel(), mesh_wo_edges[0][:,:,-1].shape), mesh_wo_edges[0][:,:,-1])

In [ ]:
alpha_from_y = (u_i_y - ion_velocity_term_1[1])/ b_field_norm[1]
alpha_from_z = (u_i_z - ion_velocity_term_1[2])/ b_field_norm[2]

alpha_from_y_flattened_z04 = alpha_from_y[:,:,-1].ravel()
alpha_from_z_flattened_z04 = alpha_from_z[:,:,-1].ravel()
points_x = mesh_wo_edges[0][:, :, -1].ravel()
points_y = mesh_wo_edges[1][:, :, -1].ravel()
points_z_z04 = mesh[2][0,0,-1]*np.ones(points_x.shape)
std_z04 = np.expand_dims(np.zeros(points_x.shape), 0)

data_y_z04 = {'a_out': np.expand_dims(alpha_from_y_flattened_z04, 0),
              'x_out': points_x,
              'y_out': points_y,
              'z_out': points_z_z04,
              'std': std_z04
              }
data_z_z04 = {'a_out': np.expand_dims(alpha_from_z_flattened_z04, 0),
              'x_out': points_x,
              'y_out': points_y,
              'z_out': points_z_z04,
              'std': std_z04
              }

data_y_z04 = wcf.remove_nan_points(data_y_z04)
data_z_z04 = wcf.remove_nan_points(data_z_z04)

alpha_interp_z04 = wcf.fit_z_alphas(data_z_z04, mesh_wo_edges, s=7e10)
alpha_fitted_z04 = np.repeat(np.expand_dims(alpha_interp_z04, 2), density.shape[2], 2)

alpha_from_z_flattened_z03 = alpha_from_z[:,:,53].ravel()
points_x = mesh_wo_edges[0][:, :, -1].ravel()
points_y = mesh_wo_edges[1][:, :, -1].ravel()
points_z_z03 = mesh[2][0,0,53]*np.ones(points_x.shape)
std_z03 = np.expand_dims(np.zeros(points_x.shape), 0)

data_z_z03 = {'a_out': np.expand_dims(alpha_from_z_flattened_z03, 0),
              'x_out': points_x,
              'y_out': points_y,
              'z_out': points_z_z03,
              'std': std_z03
              }

data_z_z03 = wcf.remove_nan_points(data_z_z03)

alpha_interp_z03 = wcf.fit_z_alphas(data_z_z03, mesh_wo_edges, s=8e10)
alpha_fitted_z03 = np.repeat(np.expand_dims(alpha_interp_z03, 2), density.shape[2], 2)

In [ ]:
wcf.plot_spline_data_knots(alpha_interp_z03, mesh_wo_edges[0][:,:,-1], mesh_wo_edges[1][:,:, -1], [], [])

In [ ]:
wcf.plot_spline_data_knots(alpha_interp_z04, mesh_wo_edges[0][:,:,-1], mesh_wo_edges[1][:,:, -1], [], [])

In [ ]:
u_i_from_alpha_z03 = ion_velocity_term_1[2][:,:,53] + alpha_interp_z03*b_field_norm[2][:,:,53]

In [ ]:
u_i_from_alpha_z04 = ion_velocity_term_1[2][:,:,-1] + alpha_interp_z04*b_field_norm[2][:,:,-1]

In [ ]:
wcf.plot_spline_data_knots(u_i_from_alpha_z03, mesh_wo_edges[0][:,:,-1], mesh_wo_edges[1][:,:, -1], [], [])

In [ ]:
wcf.plot_spline_data_knots(u_i_from_alpha_z04, mesh_wo_edges[0][:,:,-1], mesh_wo_edges[1][:,:, -1], [], [])

In [ ]:
wcf.plot_spline_data_knots(ion_velocity_term_1[2][:,:,-1], mesh_wo_edges[0][:,:,-1], mesh_wo_edges[1][:,:, -1], [], [])

In [ ]:
m, y0 = wcf.fit_line(0.3, 0.4, alpha_interp_z03, alpha_interp_z04)

In [ ]:
wcf.line(m, y0, np.linspace(0.24, 0.4, 100)).shape

In [ ]:
now = datetime.now().strftime("%Y-%m-%d-%H-%M")
out_dir = '../output/' + now
try:
    os.makedirs(out_dir)
except:
    pass

alpha = 1

(x_min, x_max, 
 y_min, y_max,
 z_min, z_max) = joint_mach_bdot_tp_extent()

spatial_increment = 0.001
mesh = np.meshgrid(np.linspace(x_min, x_max, np.ceil((x_max-x_min)/spatial_increment)),
                   np.linspace(y_min, y_max, np.ceil((y_max-y_min)/spatial_increment)),
                   np.linspace(z_min, z_max, np.ceil((z_max-z_min)/spatial_increment)))

mesh_wo_edges = wcf.remove_edges_mesh([np.array(mesh[0]), 
                                       np.array(mesh[1]), 
                                       np.array(mesh[2])])
ones = np.ones(mesh_wo_edges[0].shape)

print time_point
bx_interpolator = bx_interpolators[time_point]
by_interpolator = by_interpolators[time_point]
bz_interpolator = bz_interpolators[time_point]
te_interpolator = te_interpolators[time_point]
n_interpolator = n_interpolators[time_point]
    
bx_derivative = wcf.triangulate_derivatives(mesh, bx_triangulation, bx_interpolator, 
                                                increment=0.0000001)
bx_derivative = wcf.remove_edges_derivative_meshes(bx_derivative)
by_derivative = wcf.triangulate_derivatives(mesh, by_triangulation, by_interpolator, 
                                                increment=0.0000001)
by_derivative = wcf.remove_edges_derivative_meshes(by_derivative)
bz_derivative = wcf.triangulate_derivatives(mesh, bz_triangulation, bz_interpolator, 
                                                increment=0.0000001)
bz_derivative = wcf.remove_edges_derivative_meshes(bz_derivative)


current = wcf.current_on_mesh([bx_derivative, 
                               by_derivative, 
                               bz_derivative])
b_field, b_field_norm = wcf.b_field_on_mesh([bx_interpolator, 
                                             by_interpolator,
                                             bz_interpolator], 
                                             mesh_wo_edges, 
                                             bias=2e-2)
    
temperature = wcf.scalar_on_mesh(te_interpolator, mesh_wo_edges)
density = wcf.scalar_on_mesh(n_interpolator, mesh_wo_edges)

# Joint quantities interpolation

In [51]:
filter_width = 15

In [52]:
now = datetime.now().strftime("%Y-%m-%d-%H-%M")
out_dir = '../output/' + now
try:
    os.makedirs(out_dir)
except:
    pass

alpha = 1

(x_min, x_max, 
 y_min, y_max,
 z_min, z_max) = joint_mach_bdot_tp_extent()

spatial_increment = 0.001
mesh = np.meshgrid(np.linspace(x_min, x_max, np.ceil((x_max-x_min)/spatial_increment)),
                   np.linspace(y_min, y_max, np.ceil((y_max-y_min)/spatial_increment)),
                   np.linspace(z_min, z_max, np.ceil((z_max-z_min)/spatial_increment)))

mesh_wo_edges = wcf.remove_edges_mesh([np.array(mesh[0]), 
                                       np.array(mesh[1]), 
                                       np.array(mesh[2])])
ones = np.ones(mesh_wo_edges[0].shape)

quantity_names = ['B_x', 'B_y', 'B_z',
                  'B_norm_x', 'B_norm_y', 'B_norm_z',
                  'j_x', 'j_y', 'j_z', 'n', 'Te',
                  'u_i_term1_x', 'u_i_term1_y', 'u_i_term1_z',
                  'u_e_norm_x', 'u_e_norm_y', 'u_e_norm_z',
                  'w_i_term1_x', 'w_i_term1_y', 'w_i_term1_z',
                  'w_i_term2_x', 'w_i_term2_y', 'w_i_term2_z',
                  'u_i_term1_x_constant_density', 'u_i_term1_y_constant_density', 'u_i_term1_z_constant_density',
                  'w_i_term1_x_constant_density', 'w_i_term1_y_constant_density', 'w_i_term1_z_constant_density',
                  'ones', 
                  'u_e_x_fitted_alpha', 'u_e_y_fitted_alpha', 'u_e_z_fitted_alpha',
                  'w_i_term2_x_fitted_alpha', 'w_i_term2_y_fitted_alpha', 'w_i_term2_z_fitted_alpha',
                  'alpha_fitted']

for time_point in xrange(len(bx_interpolators)):
    print time_point
    bx_interpolator = bx_interpolators[time_point]
    by_interpolator = by_interpolators[time_point]
    bz_interpolator = bz_interpolators[time_point]
    te_interpolator = te_interpolators[time_point]
    n_interpolator = n_interpolators[time_point]
    
    bx_derivative = wcf.triangulate_derivatives(mesh, bx_triangulation, bx_interpolator, 
                                                increment=0.0000001)
    bx_derivative = wcf.remove_edges_derivative_meshes(bx_derivative)
    by_derivative = wcf.triangulate_derivatives(mesh, by_triangulation, by_interpolator, 
                                                increment=0.0000001)
    by_derivative = wcf.remove_edges_derivative_meshes(by_derivative)
    bz_derivative = wcf.triangulate_derivatives(mesh, bz_triangulation, bz_interpolator, 
                                                increment=0.0000001)
    bz_derivative = wcf.remove_edges_derivative_meshes(bz_derivative)


    current = wcf.current_on_mesh([bx_derivative, 
                                   by_derivative, 
                                   bz_derivative])
    b_field, b_field_norm = wcf.b_field_on_mesh([bx_interpolator, 
                                                 by_interpolator,
                                                 bz_interpolator], 
                                                mesh_wo_edges, 
                                                bias=2e-2)
    
    temperature = wcf.scalar_on_mesh(te_interpolator, mesh_wo_edges)
    density = wcf.scalar_on_mesh(n_interpolator, mesh_wo_edges)

    current = np.asarray(current)
    density = np.asarray(density)
    b_field_norm = np.asarray(b_field_norm)

    density = wcf.boxcar_filter_quantity_mesh(density, filter_width)

    for direction in xrange(len(current)):
        current[direction] = wcf.boxcar_filter_quantity_mesh(current[direction], 
                                                             filter_width)

    density_constant = 1e18*np.ones(density.shape)
        
    ion_velocity_term_1 = wcf.calc_ion_velocity_term_1(current, density, q_e)
    ion_velocity_term_1_constant_density = wcf.calc_ion_velocity_term_1(current, 
                                                                        density_constant, q_e)
    
    
    mach_y_interpolator = mach_y_interpolators[time_point]
    mach_z_interpolator = mach_z_interpolators[time_point]
 
    mach_y = wcf.scalar_on_mesh(mach_y_interpolator, mesh_wo_edges[:2])
    mach_z = wcf.scalar_on_mesh(mach_z_interpolator, mesh_wo_edges)
    te = wcf.scalar_on_mesh(te_interpolator, mesh_wo_edges)

    u_i_y = np.sqrt(te*q_e/m_i)*mach_y
    u_i_z = np.sqrt(te*q_e/m_i)*mach_z

    u_i_y = np.reshape(u_i_y, mesh_wo_edges[0].shape)
    u_i_z = np.reshape(u_i_z, mesh_wo_edges[0].shape)
    
    alpha_from_y = (u_i_y - ion_velocity_term_1[1])/ b_field_norm[1]
    alpha_from_z = (u_i_z - ion_velocity_term_1[2])/ b_field_norm[2]

    alpha_from_y_flattened = alpha_from_y[:,:,-1].ravel()
    alpha_from_z_flattened = alpha_from_z[:,:,-1].ravel()
    points_x = mesh_wo_edges[0][:, :, -1].ravel()
    points_y = mesh_wo_edges[1][:, :, -1].ravel()
    points_z = mesh[2][0,0,-1]*np.ones(points_x.shape)
    std = np.expand_dims(np.zeros(points_x.shape), 0)

    data_y = {'a_out': np.expand_dims(alpha_from_y_flattened, 0), 
              'x_out': points_x,
              'y_out': points_y,
              'z_out': points_z,
              'std': std
             }
    data_z = {'a_out': np.expand_dims(alpha_from_z_flattened, 0),
              'x_out': points_x,
              'y_out': points_y,
              'z_out': points_z,
              'std': std
              }

    data_y = wcf.remove_nan_points(data_y)
    data_z = wcf.remove_nan_points(data_z)
    
    alpha_interp = wcf.fit_z_alphas(data_z, mesh_wo_edges, s=7e10)   
    alpha_fitted = np.repeat(np.expand_dims(alpha_interp, 2), density.shape[2], 2)

    ion_velocity_term_2 = wcf.calc_ion_velocity_term_2(b_field_norm, alpha)
    ion_velocity_term_2_alpha = wcf.calc_ion_velocity_term_2(b_field_norm, alpha_fitted)
    
    ion_vorticity_term_1 = wcf.calc_ion_vorticity_term_1(current, density, q_e, mesh_wo_edges)
    ion_vorticity_term_1_constant_density = wcf.calc_ion_vorticity_term_1(current, density_constant, 
                                                                          q_e, mesh_wo_edges)
    ion_vorticity_term_2 = wcf.calc_ion_vorticity_term_2(b_field_norm, alpha, mesh_wo_edges)
    ion_vorticity_term_2_alpha = wcf.alc_ion_vorticity_term_2(b_field_norm, alpha_fitted, mesh_wo_edges)
    
    

    for direction in xrange(len(ion_vorticity_term_1)):
        ion_vorticity_term_1[direction] = wcf.boxcar_filter_quantity_mesh(ion_vorticity_term_1[direction], 
                                                                          filter_width)
        ion_vorticity_term_1_constant_density[direction] = wcf.boxcar_filter_quantity_mesh(ion_vorticity_term_1_constant_density[direction], 
                                                                                           filter_width)
        ion_vorticity_term_2[direction] = wcf.boxcar_filter_quantity_mesh(ion_vorticity_term_2[direction], filter_width)
        ion_vorticity_term_2_alpha[direction] = wcf.boxcar_filter_quantity_mesh(ion_vorticity_term_2_alpha[direction], 
                                                                                filter_width)
      
    fields = (list(b_field) + list(b_field_norm) + list(current) + 
              [density] + [temperature] +
              list(ion_velocity_term_1) + list(ion_velocity_term_2) +
              list(ion_vorticity_term_1) + list(ion_vorticity_term_2) +
              list(ion_velocity_term_1_constant_density) +
              list(ion_vorticity_term_1_constant_density) +
              [ones] + 
              list(ion_velocity_term_2_alpha)+
              list(ion_vorticity_term_2_alpha) +
              [alpha_fitted])

    numpy_archive_name = out_dir + '/Bdot_triple_probe_quantities' + str(time_point).zfill(4) + '.npz'
    save_to_numpy_mesh(mesh_wo_edges, fields[5:9], quantity_names[5:9], numpy_archive_name)    
    
    x, y, z, variables = wcf.prepare_for_rectilinear_grid(mesh_wo_edges, fields,
                                                          quantity_names)

    wcf.write_fields_and_currents_to_structured_mesh(now, 'Bdot_triple_probe_quantities', 
                                                     x, y, z, variables, time_point)

0

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0000
1

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0001
2

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0002
3



/Users/vonderlinden2/anaconda/lib/python2.7/site-packages/scipy/interpolate/fitpack2.py:1033: UserWarning: ier=12049
  warnings.warn(message)


../output/2016-09-22-16-53/Bdot_triple_probe_quantities0003
4

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0004
5

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0005
6

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0006
7

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0007
8

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0008
9

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0009
10

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0010
11

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0011
12

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0012
13

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0013
14

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0014
15

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0015
16

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0016
17

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0017
18

../output/2016-09-22-16-53/Bdot_triple_probe_q

/Users/vonderlinden2/anaconda/lib/python2.7/site-packages/scipy/interpolate/fitpack2.py:1033: UserWarning: ier=13620
  warnings.warn(message)


../output/2016-09-22-16-53/Bdot_triple_probe_quantities0219
220

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0220
221

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0221
222

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0222
223

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0223
224

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0224
225

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0225
226

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0226
227



/Users/vonderlinden2/anaconda/lib/python2.7/site-packages/scipy/interpolate/fitpack2.py:1033: UserWarning: ier=12799
  warnings.warn(message)


../output/2016-09-22-16-53/Bdot_triple_probe_quantities0227
228

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0228
229

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0229
230

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0230
231

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0231
232

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0232
233

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0233
234

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0234
235

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0235
236

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0236
237

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0237
238

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0238
239

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0239
240

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0240
241

../output/2016-09-22-16-53/Bdot_triple_probe_quantities0241
242

../output/2016-09-22-16-5

In [69]:
print 'density between', density.max(), density.min(), density.mean()
print 'abs density between', np.abs(density).max(), np.abs(density).min(), np.abs(density).mean()
print 'charge', q_e
print 'current x between', current[0].max(), current[0].min(), current[0].mean()
print 'current y between', current[1].max(), current[1].min(), current[1].mean()
print 'current z between', current[2].max(), current[2].min(), current[2].mean()
denominator = density*q_e
print 'denominator between', denominator.max(), denominator.min(), denominator.mean()
print 'abs denominator between', np.abs(denominator).max(), np.abs(denominator).min(), np.abs(denominator).mean()
term = [current[0]/denominator,
        current[1]/denominator,
        current[2]/denominator]
print 'term x between', term[0].max(), term[0].min(), term[0].mean()
print 'term y between', term[1].max(), term[1].min(), term[1].mean()
print 'term z between', term[2].max(), term[2].min(), term[2].mean()

density between 7.18761714797e+19 1.26165033215e+17 7.67230104918e+18
abs density between 7.18761714797e+19 1.26165033215e+17 7.67230104918e+18
charge 1.6021766208e-19
current x between 1445823.091 -664716.029921 287.957349552
current y between 1715203.22622 -1893650.19427 -3720.90718255
current z between 3060044.45744 -2740383.91419 -28779.1816356
denominator between 11.5158321537 0.020213866658 1.22923813687
abs denominator between 11.5158321537 0.020213866658 1.22923813687
term x between 13140771.3648 -4042828.16287 1798.77853144
term y between 8944163.4699 -8274999.82743 -4840.150886
term z between 7991128.24954 -7980696.62142 -10766.3934199
